In [1]:
#  các thư viện
# pip install -r requirements.txt

In [1]:
import os
import sys
import json
import argparse
# from collectionamespace import defaultdict, Counter, deque
import mwparserfromhell  
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [2]:
# TẬP HẠT GIỐNG
SEED_LIST_FILE = "seed_list.txt"

XML_FILE = "viwiki-latest-pages-articles.xml"

# ĐỌC FILE TẬP HẠT GIỐNG

## read_seeds

In [3]:
def read_seeds(path):
    '''Đọc danh sách từ file, mỗi dòng là 1 trang wikipedia.'''
    seed_list = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Bỏ qua dòng trống
                seed_list.append(line)
    return seed_list

# test

print(read_seeds(SEED_LIST_FILE))

['Trấn Thành', 'Ninh Dương Lan Ngọc', 'Ngô Thanh Vân', 'Hồng Đào', 'Kiều Minh Tuấn', 'Victor Vũ', 'Charlie Nguyễn', 'Vũ Ngọc Đãng', 'Kaity Nguyễn', 'Jun Vũ']


# ĐỌC TEXT TỪ FILE XML WIKIPEDIA

In [4]:
#test, 30 line
def show_xml_content(xml_path, n_lines=30):
    """
    In ra n_lines dòng đầu của file XML để xem cấu trúc.
    """
    with open(xml_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(line.rstrip())
            if i >= n_lines - 1:
                break


show_xml_content("viwiki-latest-pages-articles.xml", n_lines=40)


<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.11/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.11/ http://www.mediawiki.org/xml/export-0.11.xsd" version="0.11" xml:lang="vi">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>viwiki</dbname>
    <base>https://vi.wikipedia.org/wiki/Trang_Ch%C3%ADnh</base>
    <generator>MediaWiki 1.45.0-wmf.20</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Phương tiện</namespace>
      <namespace key="-1" case="first-letter">Đặc biệt</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Thảo luận</namespace>
      <namespace key="2" case="first-letter">Thành viên</namespace>
      <namespace key="3" case="first-letter">Thảo luận Thành viên</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter"

In [89]:
# đọc file XML wiki dump 

# mỗi bài viết trong thẻ <page> </page> 

    # Trả về danh sách [(title, text), ...]

# dùng iterparse (đọc từng phần nhỏ)


def read_pages_from_xml(xml_path):
    """
    Đọc từng trang trong file XML (Wikipedia dump).
    Trả về danh sách [(title, text), ...]
    """
    # danh sách lưu kết quả
    pages = []  
    # iterparse => đọc
    context = ET.iterparse(xml_path, events=("start", "end"))

    # Lấy namespace (ở thẻ <mediawiki>)
    event, root = next(context)
    
    if root.tag.startswith("{"):
        namespace = root.tag.split("}")[0].strip("{")
        namespace_prefix = f"{{{namespace}}}"
    else:
        namespace_prefix = ""

    
    # Duyệt từng p.tử trg XML
    for event, element in context:
        if event == "end" and element.tag == namespace_prefix + "page":
            title_element = element.find(f"./{namespace_prefix}title")
            text_element = element.find(f".//{namespace_prefix}text")
            
            # Lấy nd trg các thẻ, nếu k có thì None 
            if title_element is not None:
                title = title_element.text
            else:
                title = None

            if text_element is not None:
                text = text_element.text
            else:
                text = ''

            pages.append((title, text))  # thêm vào danh sách

            element.clear()  # xóa khỏi bộ nhớ

    return pages



In [6]:
# test in 3 bai
for i, (title, text) in enumerate(read_pages_from_xml(XML_FILE)):
    print(f"{i+1}. {title}")
    print(text[:300])  # chỉ in 300 ký tự đầu
    print("===============")
    if i == 2:  # dừng sau 3 bài đầu
        break


1. Alexandre De Rhodes
#đổi [[Alexandre de Rhodes]]
2. Trang Chính
<templatestyles src="Wiki2021/styles.css" />__NOTOC____NOEDITSECTION__
<div class="mainpage">
{{Wiki2021/Đầu trang}}
<div class="main-body">
    <div class="main-body__left">
{{Wiki2021/Khung
| id = feature-article
| explore card = tfa
| icon modifier = -icon-star
| icon box color modifier = -icon-b
3. Internet Society
{{đổi hướng đến đây|ISOC}}
{{Hộp thông tin tổ chức
| name            = Internet Society<br/>ISOC
| logo            = Internet Society logo.svg
| alt             = Biểu trưng Internet Society
| abbreviation    = ISOC
| founders        = [[Vint Cerf]], [[Bob Kahn]]
| tax_id          = 54-1650477<ref n


In [7]:
all_pages = read_pages_from_xml(XML_FILE)

wiki_dict = {}
for title, text in all_pages:
    if title:
        wiki_dict[title] = text

def get_wikitext(title):
    """
    Lấy nội dung wikitext từ title
    k tìm thấy, trả về chuỗi rỗng
    """
    if title in wiki_dict:
        return wiki_dict[title]
    else:
        return ""


In [8]:
from datetime import datetime

def generate_date_filename():
    now = datetime.now()
    filename = now.strftime("%d-%m-%Y_%H-%M-%S")
    return filename

print(generate_date_filename())


23-10-2025_09-14-10


In [91]:
#test
print(get_wikitext("Ninh Dương Lan Ngọc"))

# with open("test/test_wikitext.txt", "w", encoding="utf-8") as f:
#     f.write(get_wikitext("Ninh Dương Lan Ngọc"))

{{Thông tin nhân vật
| tên = Ninh Dương Lan Ngọc
| hình = Ninh Dương Lan Ngọc trong Mối Tình Đầu Của Tôi.png
| ghi chú hình = Ninh Dương Lan Ngọc vào năm 2019
| tên khai sinh = 
| ngày sinh = {{Birth date and age|1990|4|4}}
| quốc tịch = {{VIE}}
| học vấn = [[Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh]]
| công việc = {{hlist|[[Diễn viên]]}}
| năm hoạt động = 2009–nay
| con cái = 
| giải thưởng = [[#Giải thưởng và đề cử|Danh sách]]
| nơi sinh = [[Thành phố Hồ Chí Minh]], [[Việt Nam]]
| tên khác = 
| dân tộc = [[Người Việt|Kinh]]
| quê quán = [[Nam Định]], [[Việt Nam]]
| notable works = [[#Danh sách phim|Danh sách]]
| signature = Signature of Ninh Duong Lan Ngoc.svg
}}

'''Ninh Dương Lan Ngọc''' (sinh ngày 4 tháng 4 năm 1990) là một nữ [[diễn viên]] người Việt Nam. Được đánh giá là một trong những nữ diễn viên Việt Nam xuất sắc nhất trong thế hệ của mình, cô bắt đầu được biết đến qua vai diễn Nương trong bộ phim ''[[Cánh đồng bất tận (phim)|Cánh đồng bất tận]]''.<ref>{{Chú 

In [11]:
#chung

def save_to_txt(file_name , content):
    with open(file_name, "w",  encoding="utf-8") as f:
        f.write(content) 

def save_to_json(file_name , content):
    with open(file_name, "w",  encoding="utf-8") as f:
        json.dump(content, f, ensure_ascii=False, indent=4)

# TRÍCH XUẤT THÔNG TIN CƠ BẢN

## extract_infobox_from_wikitext

In [95]:
def extract_infobox_from_wikitext(wikitext):
    
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode obj
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    # duyệt qua toàn bộ cấu trúc wikitext, trả về list all các template (các khối {{ ... }})
    templates = code.filter_templates() 
   
    info = {}

    for template in templates:
        # template là obj

        # tên của template 
        name = str(template.name).lower()
        infoboxs = ["thông tin nhân vật", "thông tin phim", "thông tin diễn viên", "thông tin nghệ sĩ"]

        for infobox in infoboxs:
            if infobox in name.lower():
                # iterate params. in list tham so
                for param in template.params:
                    key = str(param.name).strip()
                    value = str(param.value).strip()
                    if key and value:
                        info[key] = value
                # break nếu k rỗng k false 
                if info:
                    break
    return info

## check_type_of_infobox

In [93]:
def check_type_of_infobox(wikitext):
    if not wikitext:
        return {}
    try:
        # biến chuỗi wikitext => Wikicode object
        code = mwparserfromhell.parse(wikitext)

    except Exception:
        return {}
    
    templates = code.filter_templates() 

    person_keys = [
        "thông tin nhân vật", "thông tin diễn viên", "thông tin nghệ sĩ"
    ]
    for template in templates:
        name = str(template.name).lower()
        if "thông tin phim" in name.lower():
            return "film"
        for key in person_keys:
            if key in name.lower(): return "person"
    return ""

In [94]:
#test
ndln_wikitext = get_wikitext("Ninh Dương Lan Ngọc")

print(extract_infobox_from_wikitext(ndln_wikitext))

# save_to_json("test/test_infobox.json" ,extract_infobox_from_wikitext(ndln_wikitext) )

{'tên': 'Ninh Dương Lan Ngọc', 'hình': 'Ninh Dương Lan Ngọc trong Mối Tình Đầu Của Tôi.png', 'ghi chú hình': 'Ninh Dương Lan Ngọc vào năm 2019', 'ngày sinh': '{{Birth date and age|1990|4|4}}', 'quốc tịch': '{{VIE}}', 'học vấn': '[[Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh]]', 'công việc': '{{hlist|[[Diễn viên]]}}', 'năm hoạt động': '2009–nay', 'giải thưởng': '[[#Giải thưởng và đề cử|Danh sách]]', 'nơi sinh': '[[Thành phố Hồ Chí Minh]], [[Việt Nam]]', 'dân tộc': '[[Người Việt|Kinh]]', 'quê quán': '[[Nam Định]], [[Việt Nam]]', 'notable works': '[[#Danh sách phim|Danh sách]]', 'signature': 'Signature of Ninh Duong Lan Ngoc.svg'}


In [96]:
def extract_wikilinks_from_wikitext(text):
   
    if not text:  # Nếu nội dung rỗng thì trả về []
        return []

    try:
        wikicode = mwparserfromhell.parse(text)  # Ptích cú pháp wikitext => obj có thể duyệt
    except Exception:
        return []  # Nếu lỗi parse thì bỏ qua

    links = []  # Danh sách kết quả
    for link in wikicode.filter_wikilinks():  # Lọc ra all [[...]] trong wikitext
        title = str(link.title).strip()       # Lấy phần title trong [[title|...]]
        if title:
            links.append(title)

    return links

In [97]:
#test 
ndln_wikilink = extract_wikilinks_from_wikitext(ndln_wikitext)
print(ndln_wikilink)

# save_to_json("test/test_link.json" ,ndln_wikilink)

['Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh', 'Diễn viên', '#Giải thưởng và đề cử', 'Thành phố Hồ Chí Minh', 'Việt Nam', 'Người Việt', 'Nam Định', 'Việt Nam', '#Danh sách phim', 'diễn viên', 'Cánh đồng bất tận (phim)', 'Báo Tuổi trẻ', 'Thành phố Hồ Chí Minh', 'Trường Đại học Sân khấu - Điện ảnh Thành phố Hồ Chí Minh', 'VnExpress', 'quảng cáo', 'ca sĩ', 'Gia đình phép thuật', 'Cánh đồng bất tận (phim)', 'Cánh Diều Vàng', 'Ngô Thanh Vân', 'Bước nhảy hoàn vũ', 'Dân Trí', 'Cô Ba Sài Gòn', 'Tấm Cám: Chuyện chưa kể', 'Gái già lắm chiêu 3', 'Cua lại vợ bầu', 'Gái già lắm chiêu 3', 'Chạy đi chờ chi', 'Chị đẹp đạp gió rẽ sóng', 'VTV3', 'Chị đẹp đạp gió rẽ sóng', 'Đạp gió 2024', 'Cao Thái Sơn', 'Only C', 'Đông Nhi', 'Cô Ba Sài Gòn', 'Jun Phạm', 'Cô Ba Sài Gòn', 'S.T Sơn Thạch', 'S.T Sơn Thạch', 'Cô Ba Sài Gòn', 'Tập đoàn LG', 'Rhymastic', 'Rhymastic', 'ZaloPay', 'Only C', 'Only C', 'Liên Quân', 'Lazada Việt Nam', 'S.T Sơn Thạch', 'Trang Pháp', 'Gia đình phép thuật', 'Mệnh lệnh Hoa

In [98]:
#test 

matBiec_wikitext = get_wikitext("Mắt biếc (phim)")
print(matBiec_wikitext)
# save_to_txt("test/test_film_wikitext.txt", matBiec_wikitext)

matBiec_infobox = extract_infobox_from_wikitext(matBiec_wikitext)
print("type mat biec infobox",type(matBiec_infobox))
# save_to_json("test/test_film_infobox.json", matBiec_infobox)

matBiec_wikilink = extract_wikilinks_from_wikitext(matBiec_wikitext)
print("type cua wikilink mat Biec", type(matBiec_wikilink))
# save_to_json("test/test_film_wikilink.json", matBiec_wikilink)

{{for|tiểu thuyết cùng tên|Mắt biếc (tiểu thuyết)}}
{{Thông tin phim
| tên = Mắt biếc
| hình = Áp phích phim Mắt biếc.jpg
| caption = Áp phích chiếu rạp của phim tại Việt Nam
| đạo diễn = [[Victor Vũ]]
| sản xuất = {{Liệt kê|
* Nguyễn Quốc Công
* Kay Nguyễn
}}
| kịch bản = {{Liệt kê|
* Victor Vũ
* A Type Machine
}}
| dựa trên = ''[[Mắt biếc (tiểu thuyết)|Mắt biếc]]'' của [[Nguyễn Nhật Ánh]]
| diễn viên = {{Liệt kê|
* [[Trần Nghĩa (diễn viên)|Trần Nghĩa]]
* [[Trúc Anh]]
* Trần Phong
* Khánh Vân
* Thảo Tâm
}}
| âm nhạc = [[Christopher Wong]]<br>[[Phan Mạnh Quỳnh]]
| quay phim = [[Dominic Pereira]]<ref>{{chú thích web|url=https://danviet.vn/nhung-guong-mat-nuoc-ngoai-thanh-cong-voi-dien-anh-viet-20220916084332568.htm|title=Những gương mặt nước ngoài thành công với điện ảnh Việt|author=Thủy Vũ|date=2022-09-16|publisher=[[Dân Việt (báo)|Dân Việt]]|access-date=2024-05-19}}</ref>
| dựng phim = Vũ Hoàng Anh
| studio = [[Galaxy EE|Galaxy M&E]]<br>November Films
| phát hành = [[Galaxy Studio]]
|

Wikipedia không nói rõ đâu là phim, đâu là người
chỉ tiêu đề + cú pháp [[...]]

phân loại liên kết -> người / phim

In [18]:
# check xem 1 link là phim or k 
def is_film_wikilink(link_title):
    if link_title is None or link_title.strip() == "":
        return False
    
    link_title_lower = link_title.lower()

    # keyword thg có trong title phim
    film_keywords = [
        "(phim",
    ]

    for keyword in film_keywords:
        if keyword in link_title_lower:
            return True
        else: 
            # tức là link này k có dấu hiệu nhận biết tại tiêu đề
            # get text ra để check infobox 
            wikitext = get_wikitext(link_title)
            if (check_type_of_infobox(wikitext) == "film"):
                # print(wikitext)
                return True


        
    return False


## is_vn_person_wikilink

In [104]:
# check 1 link la person or k

def is_vn_person_wikilink(link_title):
    if link_title is None or link_title.strip() == "":
        return False
    
    link_title_lower = link_title.lower()

    # keyword thg có trong title phim
    film_keywords = [
        "(diễn viên",
    ]

   
    for keyword in film_keywords:
        if keyword in link_title_lower:
            return True
        else: 
            # link này k có dấu hiệu nhận biết tại title
            # get text ra để check infobox 
            wikitext = get_wikitext(link_title)
            if (check_type_of_infobox(wikitext) == "person"):
          
                infobox = extract_infobox_from_wikitext(wikitext)
               
                # gom hết text trong infobox để tìm
                text_all = " ".join(str(v) for v in infobox.values()).lower()
                # print(text_all)
            
                if any(x in text_all for x in ["việt nam", "{{vie}}", "vietnam"]):
                    # print("=> Người Việt Nam")
                    return True
                
                

 
    return False

#test

print(is_vn_person_wikilink("Ninh Dương Lan Ngọc"))
print(is_vn_person_wikilink("Trần Nghĩa (diễn viên)"))
print(is_vn_person_wikilink("Trúc Anh"))
print(is_vn_person_wikilink("Victor Vũ"))
print(is_vn_person_wikilink("Thanh Hiền"))

True
True
True
True
True


In [76]:
print(get_wikitext("Thanh Hiền"))

{{Về|diễn viên sinh năm 1954}}{{Thông tin nghệ sĩ
| tên = Thanh Hiền
| nền = diễn viên
| tên khai sinh = Trần Thị Thanh Hiền
| ngày sinh = 04/08/1954
| quốc tịch = {{VIE}}
| nghề nghiệp = [[Diễn viên]]
| phim = [[Lật mặt 7: Một điều ước]]
| dân tộc = [[Người Việt|Kinh]]
}}

'''Thanh Hiền''', tên thật là '''Trần Thị Thanh Hiền''' (sinh năm 1954) là một [[nữ diễn viên]] người [[Việt Nam]]. Bắt đầu sự nghiệp diễn xuất từ năm 51 tuổi, sau 20 năm, bà đã tham gia hơn 700 dự án điện ảnh và truyền hình.<ref>{{Chú thích web|url=https://vietnamnet.vn/thanh-hien-lat-mat-7-nu-chinh-lon-tuoi-nhat-man-anh-nhieu-lan-bi-quyt-cat-se-2275569.html|tiêu đề=Thanh Hiền 'Lật mặt 7': Nữ chính lớn tuổi nhất màn ảnh, hạnh phúc bên chồng con|tác giả=Tuấn Chiêu|ngày=2024-05-01|nhà xuất bản=[[VietNamNet]]|ngôn ngữ=vi|ngày truy cập=2024-05-04}}</ref> Thanh Hiền được biết đến qua bộ [[phim điện ảnh]] ''[[Lật mặt 7: Một điều ước]]'' của [[Đạo diễn điện ảnh|đạo diễn]] [[Lý Hải]] năm [[2024]].<ref>{{Chú thích web|url=h

In [24]:
print(get_wikitext("Ninh Dương Lan Ngọc"))

{{Thông tin nhân vật
| tên = Ninh Dương Lan Ngọc
| hình = Ninh Dương Lan Ngọc trong Mối Tình Đầu Của Tôi.png
| ghi chú hình = Ninh Dương Lan Ngọc vào năm 2019
| tên khai sinh = 
| ngày sinh = {{Birth date and age|1990|4|4}}
| quốc tịch = {{VIE}}
| học vấn = [[Trường Đại học Sân khấu – Điện ảnh Thành phố Hồ Chí Minh]]
| công việc = {{hlist|[[Diễn viên]]}}
| năm hoạt động = 2009–nay
| con cái = 
| giải thưởng = [[#Giải thưởng và đề cử|Danh sách]]
| nơi sinh = [[Thành phố Hồ Chí Minh]], [[Việt Nam]]
| tên khác = 
| dân tộc = [[Người Việt|Kinh]]
| quê quán = [[Nam Định]], [[Việt Nam]]
| notable works = [[#Danh sách phim|Danh sách]]
| signature = Signature of Ninh Duong Lan Ngoc.svg
}}

'''Ninh Dương Lan Ngọc''' (sinh ngày 4 tháng 4 năm 1990) là một nữ [[diễn viên]] người Việt Nam. Được đánh giá là một trong những nữ diễn viên Việt Nam xuất sắc nhất trong thế hệ của mình, cô bắt đầu được biết đến qua vai diễn Nương trong bộ phim ''[[Cánh đồng bất tận (phim)|Cánh đồng bất tận]]''.<ref>{{Chú 

In [47]:
# print(get_wikitext("Trúc Anh"))
print(check_type_of_infobox(get_wikitext("Trúc Anh")))


person


## unique_list

In [99]:
#chung
# Bỏ lặp, giữ đúng thứ tự 

def unique_list(items_list):

    seen_items = set()   # chứa các phần tử đã gặp
    result = []          # danh sách kết quả

    for item in items_list:
        if item not in seen_items:
            result.append(item)    # thêm vào kết quả
            seen_items.add(item)   # đánh dấu là đã gặp

    return result


#test 
print(unique_list(["Trấn Thành", "Ngô Thanh Vân", "Trấn Thành"]))

['Trấn Thành', 'Ngô Thanh Vân']


In [60]:
test_wikilink_list = []
for l in ndln_wikilink:
    if is_film_wikilink(l): 
        print(l)
        test_wikilink_list.append(l)

test_wikilink_list = unique_list(test_wikilink_list)
print(len(test_wikilink_list))

Cánh đồng bất tận (phim)
Cánh đồng bất tận (phim)
Cô Ba Sài Gòn
Tấm Cám: Chuyện chưa kể
Gái già lắm chiêu 3
Cua lại vợ bầu
Gái già lắm chiêu 3
Cô Ba Sài Gòn
Cô Ba Sài Gòn
Cô Ba Sài Gòn
Nguyệt thực (phim truyền hình)
Cánh đồng bất tận (phim)
Trúng số (phim 2015)
Tấm Cám: Chuyện chưa kể
Cô Ba Sài Gòn
Cua lại vợ bầu
Gái già lắm chiêu 3
Gái già lắm chiêu V: Những cuộc đời vương giả
Cô gái từ quá khứ
Cánh đồng bất tận (phim)
Trúng số (phim 2015)
Cô Ba Sài Gòn
Cánh đồng bất tận (phim)
Cô Ba Sài Gòn
Cánh đồng bất tận (phim)
Trúng số (phim 2015)
Tấm Cám: Chuyện chưa kể
Cánh đồng bất tận (phim)
Tấm Cám: Chuyện chưa kể
Cua lại vợ bầu
Gái già lắm chiêu 3
Cô gái từ quá khứ
Tấm Cám: Chuyện chưa kể
Cô Ba Sài Gòn
Gái già lắm chiêu 3
Cô gái từ quá khứ
Cô gái từ quá khứ
Cô gái từ quá khứ
9


## extract_film_links_from_person_text

In [100]:
def extract_film_links_from_person_text(wikitext):

    # Lấy tất cả liên kết nội bộ trong bài
    all_wikilinks = extract_wikilinks_from_wikitext(wikitext)

    film_links = []   # danh sách phim tìm được

    # Bỏ trùng, giữ thứ tự
    unique_links = unique_list(all_wikilinks)

    # Kiểm tra từng link
    for link in unique_links:
        if is_film_wikilink(link):
            film_links.append(link)

    return film_links

In [62]:
#test
print(extract_film_links_from_person_text(ndln_wikitext))
print(len(extract_film_links_from_person_text(ndln_wikitext)))

['Cánh đồng bất tận (phim)', 'Cô Ba Sài Gòn', 'Tấm Cám: Chuyện chưa kể', 'Gái già lắm chiêu 3', 'Cua lại vợ bầu', 'Nguyệt thực (phim truyền hình)', 'Trúng số (phim 2015)', 'Gái già lắm chiêu V: Những cuộc đời vương giả', 'Cô gái từ quá khứ']
9


wikifilm -> person 

## extract_cast_section

In [65]:
def extract_cast_section(wikitext):
    text_lower = wikitext.lower()

    # tìm vị trí tiêu đề '== diễn viên =='
    start_position = text_lower.find("== diễn viên ==")
    if start_position == -1: return "" # str rỗng, ko có 

    # cắt phần vb sau title dvien
    content = wikitext[start_position + len("== Diễn viên =="):] 

    # tìm vị trí tiêu đề tiếp theo (== ... ==)
    end_position = content.find("==")
    if end_position != -1: section = content[:end_position] 
    else: section = content # k có title next -> lay het 

    return section.strip() # delete space thừa

print(extract_cast_section(matBiec_wikitext))

[[Tập tin:Mắt biếc – Behind the scenes (2).png|nhỏ|Dàn diễn viên phim ''Mắt biếc'']]
*[[Trần Nghĩa (diễn viên)|Trần Nghĩa]] vai Ngạn:<ref>{{Chú thích web|url=http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|title=Victor Vũ truy lùng nàng Mắt Biếc và chàng si tình|last=|first=|date=5 tháng 11 năm 2018|website=Báo Đất Việt|archive-url=https://web.archive.org/web/20190716065358/http://baodatviet.vn/van-hoa/tin-tuc-giai-tri/victor-vu-truy-lung-nang-mat-biec-va-chang-si-tinh-3368589/|archive-date=2019-07-16|url-status=dead|access-date=20 tháng 1 năm 2019}}</ref><ref>{{chú thích web|url=https://ione.vnexpress.net/tin-tuc/phim/vietnam/vuot-qua-hon-1-400-ung-vien-mat-biec-cua-victor-vu-da-lo-dien-3886413.html|website=IONE.VNEXPRESS.NET|tiêu đề=Vượt qua hơn 1.400 ứng viên, 'Mắt Biếc' của Victor Vũ đã lộ diện|ngày truy cập=2019-02-27|ngày=2019-02-26|ngôn ngữ=vi}}</ref> Chàng trai sinh ra và lớn lên ở làng Đo Đo, mang dáng vẻ thư sinh cùng 

In [ ]:
#test

nhaBaNu = get_wikitext("Nhà bà Nữ")

## extract_cast_list

In [108]:
import re
def extract_cast_list(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    cast_list = []

    # lay all dòng begin * :*[[Trúc Anh]]
    lines = []
    # duyệt từng node => chuyển node thành chuỗi, bỏ khoảng trắng thừa => if line begin * -> add vào lines

    # default 
    actor_name_list = []
    character_list = []
    
    for node in wikicode.nodes:  

        text = str(node).strip()  

        if ("*" in text) or (not text):
            continue
        
        # lấy những line begin [[ , vai 
        # [[Trúc
        # vai Hà Lan
        if text.startswith('[[Tập tin:'): continue       
        if text.startswith('[['):       
            if ("[[" in text) and ("]]" in text):
                text = text.replace("[[", "").replace("]]", "")
                if "|" in text:
                    text = text.split("|")[0].strip()      
            actor_name_list.append(text)

        if "vai" in text:
            # tim kết quả first = search cụm begin 'vai' + tên vai (tới "," or ".")             
            match = re.search(r"vai\s+([^,.:]+)", text)
          
            if match: 
                text = match.group(1).strip()
                character_list.append(text)
            else: print("Không tìm thấy vai")


    for actor, character in zip(actor_name_list, character_list):
        actor_obj = {
            "actor_name": actor,
            "character": character
        }
        # add list
        cast_list.append(actor_obj)

    
    return cast_list


#test 

print(extract_cast_list(extract_cast_section(matBiec_wikitext)))



[{'actor_name': 'Trần Nghĩa (diễn viên)', 'character': 'Ngạn'}, {'actor_name': 'Trúc Anh', 'character': 'Hà Lan'}, {'actor_name': 'Khánh Huyền', 'character': 'Dũng'}, {'actor_name': 'Thanh Hiền', 'character': 'Trà Long'}]


In [75]:
#test 

print(extract_cast_list(extract_cast_section(nhaBaNu)))

[{'actor_name': 'Lê Giang', 'character': 'Ngọc Nữ'}, {'actor_name': 'Uyển Ân', 'character': 'Ngọc Nhi'}, {'actor_name': 'Song Luân', 'character': 'John'}, {'actor_name': 'Trấn Thành', 'character': 'Phú Nhuận'}, {'actor_name': 'Khả Như', 'character': 'Ngọc Như'}, {'actor_name': 'Ngọc Giàu', 'character': 'Ngọc Ngà'}, {'actor_name': 'Lê Dương Bảo Lâm', 'character': 'Lê Minh Hổ'}, {'actor_name': 'Việt Anh (nghệ sĩ)', 'character': 'ông Liêm'}, {'actor_name': 'Công Ninh', 'character': 'bố của John'}, {'actor_name': 'Ngân Quỳnh', 'character': 'mẹ của John'}]


[[Tập tin:Mắt biếc – Behind the scenes (2).png|nhỏ|Dàn diễn viên phim ''Mắt biếc'']] 
 
* 
[[Trần Nghĩa (diễn viên)|Trần Nghĩa]] 
vai Ngạn: 

## extract_actors_from_film

In [102]:
def extract_actors_from_film(wikitext):
    # Trích danh sách diễn viên từ '== Diễn viên ==' của wikitext phim
    cast_section = extract_cast_section(wikitext)
    
    #list 
    cast_list = extract_cast_list(cast_section)
    # {name: , char: }


    vn_cast_list = []
    for cast in cast_list:
        actor_name = cast.get('actor_name')  # lấy tên diễn viên
        if not actor_name: continue  # bỏ qua 

        if is_vn_person_wikilink(actor_name):
            vn_cast_list.append(cast)
        
    return vn_cast_list

#test
matBiec_actors = extract_actors_from_film(matBiec_wikitext)

print(matBiec_actors)

[{'actor_name': 'Trần Nghĩa (diễn viên)', 'character': 'Ngạn'}, {'actor_name': 'Trúc Anh', 'character': 'Hà Lan'}, {'actor_name': 'Khánh Huyền', 'character': 'Cô của Hà Lan'}, {'actor_name': 'Thanh Hiền', 'character': 'Bà nội của Ngạn'}]


## extract_directors_from_film

In [88]:
def extract_directors_from_film(wikitext):
    infobox = extract_infobox_from_wikitext(wikitext)
    director = infobox.get('đạo diễn')
    if ("[[" in director) and ("]]" in director):
        director = director.replace("[[", "").replace("]]", "")

    return director
#test
print(extract_directors_from_film(matBiec_wikitext))

Victor Vũ


In [83]:
SEED_LIST = read_seeds(SEED_LIST_FILE)

print(SEED_LIST)

['Trấn Thành', 'Ninh Dương Lan Ngọc', 'Ngô Thanh Vân', 'Hồng Đào', 'Kiều Minh Tuấn', 'Victor Vũ', 'Charlie Nguyễn', 'Vũ Ngọc Đãng', 'Kaity Nguyễn', 'Jun Vũ']


In [109]:
print(extract_actors_from_film(get_wikitext("Dòng máu anh hùng")))

[{'actor_name': 'Johnny Trí Nguyễn', 'character': 'Lê Văn Cường'}, {'actor_name': 'Ngô Thanh Vân', 'character': 'Sỹ'}, {'actor_name': 'Dustin Nguyễn', 'character': 'Tham Nguyễn'}, {'actor_name': 'Nguyễn Chánh Tín', 'character': 'Hứa Danh'}, {'actor_name': 'Nguyễn Thắng', 'character': 'Sĩ quan Tessier'}, {'actor_name': 'Stephane Gauger', 'character': 'Người gián điệp'}, {'actor_name': 'Nguyễn Hậu', 'character': 'Công nhân trong mỏ sắt'}]
